In [ ]:
%matplotlib inline
import xarray as xr
import torch
from uwnet.model import MLP

from uwnet.interface import dataset_to_numpy_dict

In [ ]:
ds = xr.open_dataset("../data/processed/training.nc").isel(time=slice(0,120), z=slice(0,28))
ds

In [ ]:
mod = MLP.from_path("../models/29/1.pkl")
mod.add_forcing = True

output = mod.call_with_xr(ds, n=1)

In [ ]:
t0 = ds.isel(time=0)

d = {key: t0[key].values[np.newaxis] for key in t0.data_vars if key in mod.inputs.names + ['FSLI', 'FQT']}
mod.call_with_numpy_dict(d, dt=0)

In [ ]:
plt.pcolormesh(output['QT'][:,:,32,0].T)
plt.colorbar()
plt.figure()
plt.pcolormesh(output['SLI'][:,:,32,0].T)
plt.colorbar()

In [ ]:
output['FQTNN'][:,:,32,0].plot(x='time')
plt.figure()
output['FSLINN'][:,:,32,0].plot(x='time')

In [ ]:
PW = (output.QT * ds.layer_mass).sum('z')/1000
PW_obs = (ds.QT * ds.layer_mass).sum('z')/1000

In [ ]:
PW.mean(['x', 'y']).plot()
PW_obs.mean(['x', 'y']).plot()

In [ ]:
pw_dot = (output.FQTNN * ds.layer_mass).sum('z').mean(['x', 'y'])/1000
pw_dot.plot()